In [1]:
# !open .

In [ ]:
%load py_neo_quant_backbone.py

In [ ]:
%load py_neo_quant_basic.py

In [ ]:
%load py_neo_quant_fnguide.py

In [ ]:
%load py_neo_quant_backtest.py

In [ ]:
companies, fs_df, fr_df, iv_df, prices = load_all_data_for_backtest()

In [ ]:
price_df = prices
start_date = '2008-6'
strategy_date = get_strategy_date(start_date)
initial_money = 100000000

In [ ]:
companies = get_company_df_from_total_capital(companies, 0.6, start_date)
fs_df = fs_df.loc[companies.index]
fr_df = fr_df.loc[companies.index]
iv_df = iv_df.loc[companies.index]

In [ ]:
pd.options.display.max_rows = 60
pd.options.display.max_columns = 20
# pd.options.display.max_rows = 6000
# pd.set_option('display.max_columns', 500)

In [ ]:
def get_low_per_kospi_firm_list(start_date):
    start_date = str(start_date)
    strategy_date = get_strategy_date(start_date)
    per = low_per(strategy_date)
    new_firm_list = get_code_list_has_price(per.index, price_df, start_date)
    per = per.loc[new_firm_list]
    per = add_price_info(per)
    per = add_company_info(per)
    per = get_kospi_list(per)

    temp_df = pd.DataFrame({'price':price_df.loc[start_date][per.index]})
    temp_index = temp_df[temp_df['price'] > 5000].index
    per = per.loc[temp_index]

    return per.index

def get_low_per_firm_list(start_date, price_df, num):
    start_date = str(start_date)
    strategy_date = get_strategy_date(start_date)
    firm_list = low_per(strategy_date).index
    return get_code_list_has_price(firm_list, price_df, start_date)[:num]

def get_low_per_porfolio(start_date):
    firm_list = get_low_per_firm_list(start_date, price_df, None)
    return firm_list

def get_high_roa_firm_list(start_date, price_df, num):
    start_date = str(start_date)
    strategy_date = get_strategy_date(start_date)
    firm_list = high_roa(fr_df, strategy_date, None).index
    return get_code_list_has_price(firm_list, price_df, start_date)[:num]

def get_high_roa_porfolio(start_date):
    firm_list = get_high_roa_firm_list(start_date, price_df, None)
    return firm_list

def get_qp_portfolio(start_date):
    start_date = str(start_date)
    strategy_date = get_strategy_date(start_date)
    qp_list = qp_formula(strategy_date)
    return qp_list.index

In [ ]:
max_profit = 0.5
max_holding_days=365
total_pf_df = get_portfolio_backtest_df(get_qp_portfolio, start_date, price_df, 20, initial_money, max_profit, max_holding_days)
bt1 = get_backtest_result_from_pf(total_pf_df, price_df)

max_profit = 0.5
max_holding_days=365
total_pf_df = get_portfolio_backtest_df(get_low_per_porfolio, start_date, price_df, 20, initial_money, max_profit, max_holding_days)
bt2 = get_backtest_result_from_pf(total_pf_df, price_df)


# max_profit = 0.5
# max_holding_days=365/2
# total_pf_df = get_portfolio_backtest_df(get_low_per_porfolio, start_date, price_df, 20, initial_money, max_profit, max_holding_days)
# bt2 = get_backtest_result_from_pf(total_pf_df, price_df)

# max_profit = 0.5
# max_holding_days=365*2
# total_pf_df = get_portfolio_backtest_df(get_high_roa_porfolio, start_date, price_df, 20, initial_money, max_profit, max_holding_days)
# bt3 = get_backtest_result_from_pf(total_pf_df, price_df)

# max_profit = 0.5
# max_holding_days=365/2
# total_pf_df = get_portfolio_backtest_df(get_high_roa_porfolio, start_date, price_df, 20, initial_money, max_profit, max_holding_days)
# bt4 = get_backtest_result_from_pf(total_pf_df, price_df)

# total_pf_df = get_portfolio_backtest_df(get_low_per_kospi_firm_list, start_date, price_df, 20, initial_money, max_profit, max_holding_days)
# bt5 = get_backtest_result_from_pf(total_pf_df, price_df)


In [ ]:
my = backtest_with_code_list(my_portfolio_code_list, start_date, None)


In [ ]:
kospi_df = prices[['KOSPI']][start_date:]
kospi_df = kospi_df.fillna(method='bfill')
kospi_df['total_change_rate'] = kospi_df/kospi_df.iloc[0]-1

kosdaq_df = prices[['KOSDAQ']][start_date:]
kosdaq_df = kosdaq_df.fillna(method='bfill')
kosdaq_df['total_change_rate'] = kosdaq_df/kosdaq_df.iloc[0]-1

In [ ]:
plt.rc('font', size=10)
plt.figure(figsize=(20, 12))
plt.plot(bt1.index, bt1['total_change_rate'], label='bt1')
plt.plot(bt2.index, bt2['total_change_rate'], label='bt2')
plt.plot(my.index, my['total_change_rate'], label='my portfolio')
plt.plot(kospi_df.index, kospi_df['total_change_rate'], label='kospi', color='red',linewidth=0.5, linestyle='dashed')
plt.plot(kosdaq_df.index, kosdaq_df['total_change_rate'], label='kosdaq', color='blue',linewidth=0.5, linestyle='dashed')
plt.title('backtest')
plt.xlabel("duration")
plt.ylabel("profit")
plt.legend(loc='upper right')
plt.grid()
plt.show()